In [ ]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from skimage.segmentation import slic, felzenszwalb, quickshift, watershed, mark_boundaries
from skimage.filters import sobel, meijering, sato, frangi, hessian, gaussian, threshold_sauvola
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.measure import label
from skimage import measure
from skimage import feature
from skimage import graph
from skimage import exposure
from scipy import stats

In [ ]:
data_path = "./data/05m_chips"
labels_dir = "labels"
slope_dir = "slope"
elevation_dir = "elevation"
test_file = "18E023_68900_6125_25_0001.tif"
# test_file = "19F048_71375_7325_25_0092.tif"
# test_file = "19F048_71350_7325_25_0076.tif"

In [ ]:
gt_path = os.path.join(data_path, labels_dir, test_file)
sl_path = os.path.join(data_path, slope_dir, test_file)
el_path = os.path.join(data_path, elevation_dir, test_file)

slope = tifffile.imread(sl_path)
elevation = tifffile.imread(el_path)
pred = tifffile.imread(gt_path)
target = tifffile.imread(gt_path)

In [ ]:
def color_segments_rand(segments):
   # Generate random colors for each label
    unique_labels = np.unique(segments)  # Find all unique segment labels
    random_colors = np.array([np.random.rand(3,) for _ in unique_labels])  # Random RGB colors

    # Initialize the colored label image
    colored_labels = np.zeros((segments.shape[0], segments.shape[1], 3))

    # Assign random colors to each label
    for i, label in enumerate(unique_labels):
        colored_labels[segments == label] = random_colors[i]
    
    return colored_labels

def color_segments_majority(segments, target, min_pixels=0):
    background_label = 0

    # Output label image
    colored_output = np.zeros_like(target)

    # Assign majority (non-background) label per segment
    for seg_val in np.unique(segments):
        mask = segments == seg_val
        values = target[mask]
        values = values[values != background_label]  # exclude background

        if len(values) > min_pixels:
            unique, counts = np.unique(values[values != background_label], return_counts=True)
            if len(unique) > 0:
                majority = unique[np.argmax(counts)]
            else:
                majority = background_label

            colored_output[mask] = majority
    
    return colored_output

def hpmf(image):
    from scipy.ndimage import median_filter
    # Step 2: Apply median filter
    filterx = 11
    filtery = 11
    median_blur = median_filter(image, size=(filtery, filterx))

    # Step 3: High-pass = Original - Median
    hpmf = image - median_blur

    # Step 4: Normalize to range 0–1 (your range: -6 to +8)
    hpmf_norm = (hpmf - (-6)) / (8 - (-6))  # Equivalent to (hpmf + 6) / 14

    return hpmf_norm

def normal_map(image, smoothing_sigma=None):
    if smoothing_sigma:
        image = gaussian(image, sigma=smoothing_sigma)

    dzdy, dzdx = np.gradient(image)

    nx = -dzdx
    ny = -dzdy
    nz = np.ones_like(image)

    length = np.sqrt(nx**2 + ny**2 + nz**2)
    nx /= length
    ny /= length
    nz /= length

    normal_map = np.stack([(nx + 1) * 127.5, (ny + 1) * 127.5, (nz + 1) * 127.5], axis=-1).astype(np.float32)
    return normal_map

def skeletonize_labels(labels, bg=0, smoothing_sigma=None):
    from skimage.morphology import skeletonize
    
    if smoothing_sigma:
        image = gaussian(image, sigma=smoothing_sigma)
        
    output = np.zeros_like(labels)

    for label in np.unique(labels):
        if label != bg:
            label_mask = ~(labels != label)
            skel_label_mask = skeletonize(label_mask)
            output[skel_label_mask] = label
    
    return output
